# Monte Carlo Tree Search

In [4]:

"""
Multi Agent systems - Assignment 6
Tiddo Loos
date:08-01-2020
"""

import random
import math
import xlsxwriter


class Node:
    def __init__(self, number, value, parent, depth):
        self.number = number
        self.value = value
        self.parent = parent
        self.children = []
        self.node_depth = depth
        self.visited = 0
        self.av_value = 0
        self.ucb = math.inf


class Tree:
    def __init__(self, depth, c):
        self.tree = []
        self.depth = depth
        self.node_count = 0
        self.c = c
        self.highest_node = None
        self.visited_nodes = []
        self.root = None

    def init_tree(self):
        root = Node(0, 0, None, 0)
        self.node_count += 1
        self.tree.append(root)
        self.make_tree()

    def make_tree(self):
        for depth in range(1, self.depth):
            for node in self.tree:
                if node.node_depth == depth - 1:
                    for i in range(0, 2):
                        self.make_children(depth, node)

    def make_children(self, depth, node):
        value = self.check_leaf(depth)
        child = Node(self.node_count, value, node, depth)
        self.tree.append(child)
        node.children.append(child)
        self.node_count += 1

    def check_leaf(self, depth):
        if depth == self.depth-1:
            value = random.uniform(0, 100)
        else:
            value = 0
        return value

    def mcts(self, iterations):
        print('**** new starting node ****')
        for i in range(iterations):
            print('-----new iteration from start node-----')
            path_list = []
            path_list.append(self.root)
            next_node = self.compare_ucb(self.root.children)
            path_list.append(next_node)
            while self.not_visited(next_node) != True and self.not_visited(next_node) != self.is_not_leaf(next_node):
                #select
                next_node = self.compare_ucb(next_node.children)
                path_list.append(next_node)
            #expanded
            if next_node not in path_list:
                path_list.append(next_node)
            reward = self.simulate(next_node)
            self.backpropagate(path_list, reward)

    def not_visited(self, node):
        if node.visited == 0:
            return True

    def simulate(self, next_node):
        reward = self.roll_out(next_node)
        return reward

    def roll_out(self, node):

        rollout = 1
        next_node = node
        reward = 0
        for i in range(rollout):
            while self.is_not_leaf(next_node):
                next_node = self.compare_ucb(next_node.children)
            reward += (next_node.value/rollout)
        return reward

    def is_not_leaf(self, node):
        if node.node_depth == self.depth - 1:
            return False
        else:
            return True

    def compare_ucb(self, children):
        child_l = children[0]
        child_r = children[1]
        if child_l.ucb > child_r.ucb:
            next_node = child_l
        elif child_l.ucb == child_r.ucb:
            # mention in report
            next_node = random.choice([child_l, child_r])
        else:
            next_node = child_r
        return next_node

    def backpropagate(self, path_list, reward):
        for node in path_list:
            self.update_visited(node)
            self.update_values(node, reward)
            self.update_ucb_value(node)

    def update_visited(self, node):
        node.visited += 1
        if node not in self.visited_nodes:
            self.visited_nodes.append(node)

    def update_values(self, node, reward):
        node.value += reward
        node.av_value = node.value/node.visited

    def update_ucb_value(self, node):
        parent = node.parent
        if parent == None:
            node.ucb = 1
            return
        else:
            math_log = math.log(parent.visited)
            upperbound = self.c * ((math_log/node.visited)**1/2)
            ucb_value = node.av_value + upperbound
            node.ucb = ucb_value


In [6]:
def main(c):
    tree_depth = 13
    iterations = 5
    tree = Tree(tree_depth, c)
    tree.init_tree()
    tree.root = tree.tree[0]
    tree.mcts(iterations)
    # create new starting node
    new_start(tree)
    tree.visited_nodes = [] #to delete memorized nodes (not in the ancestor of hihghest chosen node)
    # create new starting node #indicate new starting nodes to liking
    tree.mcts(iterations)
    new_start(tree)
    tree.visited_nodes = [] #to delete memorized nodes (not in the ancestor of hihghest chosen node)
    tree.mcts(iterations)
    best_choice = node_to_move_to(tree)
    return best_choice.av_value


def new_start(tree):
    tree.root = tree.visited_nodes[0]
    for node in tree.visited_nodes:
        if node.av_value >= tree.root.av_value:
            tree.root = node
    return


def node_to_move_to(tree):
    result = tree.visited_nodes[0]
    for node in tree.visited_nodes:
        if node.av_value >= result.av_value:
            result = node
    return result


def collect_data():
    c = 0.0
    av_ucb_list = []
    c_values = []
    for i in range(100):
        print('_____________new iteration with different c value________________')
        best_choices = []
        for best_choice in range(10):
            best_choice = main(c)
            best_choices.append(best_choice)
        average_ucb = sum(best_choices)/len(best_choices)
        av_ucb_list.append(average_ucb)
        c_values.append(c)
        c += 0.05

    # create excel file
    workbook = xlsxwriter.Workbook('MCTS.xlsx')
    worksheet = workbook.add_worksheet()

    worksheet.write('A1', 'c value')
    worksheet.write('B1', 'av_value') #or UCB-value dependeing on what value you research

    row = 1
    column = 0
    for item in c_values:
        worksheet.write(row, column, item)
        row += 1
    column += 1
    row = 1
    for item in av_ucb_list:
        worksheet.write(row, column, item)
        row += 1
    workbook.close()


collect_data()

_____________new iteration with different c value________________
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

...

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
-----new iteration from start node-----
**** new starting node ****
-----new iteration from start node-----
-----new iteration f

KeyboardInterrupt: 